### Automação GTPlan

### Versão Correta

In [33]:
import pandas as pd
import glob
import os
import shutil
from datetime import datetime, timedelta

# Mapeamento manual dos nomes dos meses em português
meses_corretos = [
    "Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
    "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"
]

# Obtém o caminho da pasta Downloads do usuário atual
pasta = os.path.join(os.path.expanduser("~"), "Downloads")

# Encontrar todos os arquivos que começam com "result" e terminam em .csv
arquivos = glob.glob(os.path.join(pasta, "result*.csv"))

# Lista para armazenar os DataFrames
dfs = []

# Loop sobre os arquivos encontrados
for arquivo in arquivos:
    df = pd.read_csv(arquivo, sep=";", dtype=str, encoding="utf-8")
    dfs.append(df)

# Concatenar todos os DataFrames empilhando os dados
df_final = pd.concat(dfs, ignore_index=True)

# Salvar como CSV garantindo a codificação correta
caminho_saida = os.path.join(pasta, "scm586g_followup.csv")

# Se o arquivo já existir, removê-lo antes de salvar o novo
if os.path.exists(caminho_saida):
    os.remove(caminho_saida)

df_final.to_csv(caminho_saida, index=False, sep=";", encoding="utf-8-sig")
print(f"Arquivo consolidado salvo em: {caminho_saida}")

# Renomear o arquivo "mrp8009g_ps_list.csv" para "scm420g_suggestions.csv"
arquivo_antigo = os.path.join(pasta, "mrp8009g_ps_list.csv")
arquivo_novo = os.path.join(pasta, "scm420g_suggestions.csv")

if os.path.exists(arquivo_antigo):
    os.rename(arquivo_antigo, arquivo_novo)
    print(f"Arquivo renomeado para: {arquivo_novo}")
else:
    print("Arquivo 'mrp8009g_ps_list.csv' não encontrado na pasta.")

# 🛠 Mover os arquivos para a pasta de destino
pasta_destino_base = r"\\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025"
pasta_destino_secundaria = r"\\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN"

arquivos_para_mover = [
    "scm586g_followup.csv",
    "scm420g_suggestions.csv",
    "sku.csv",
    "sku (1).csv"
]

# Obtém a data atual
data_atual = datetime.now()
nome_mes = meses_corretos[data_atual.month - 1]

pasta_mes = os.path.join(pasta_destino_base, nome_mes)
if not os.path.exists(pasta_mes):
    os.makedirs(pasta_mes)

pasta_dia = os.path.join(pasta_mes, data_atual.strftime("%d.%m"))
if not os.path.exists(pasta_dia):
    os.makedirs(pasta_dia)

# Mover arquivos principais
temp_files = []
for arquivo in arquivos_para_mover:
    origem = os.path.join(pasta, arquivo)
    destino = os.path.join(pasta_dia, arquivo)
    
    if os.path.exists(origem):
        shutil.move(origem, destino)
        print(f"Arquivo movido para: {destino}")
        temp_files.append(destino)
    else:
        print(f"Arquivo '{arquivo}' não encontrado na pasta de Downloads.")

# Mover arquivos "result*.csv"
for arquivo in arquivos:
    nome_arquivo = os.path.basename(arquivo)
    destino = os.path.join(pasta_dia, nome_arquivo)
    shutil.move(arquivo, destino)
    print(f"Arquivo de origem movido para: {destino}")

# Mover arquivos para a pasta secundária
for arquivo in arquivos_para_mover:
    origem = os.path.join(pasta_dia, arquivo)
    destino = os.path.join(pasta_destino_secundaria, arquivo)
    
    if os.path.exists(origem):
        shutil.copy2(origem, destino)
        print(f"Arquivo copiado para: {destino}")
    else:
        print(f"Arquivo '{arquivo}' não encontrado para copiar.")

# 📌 Copiar arquivos .pbix para backup
pasta_origem_pbix = r"\\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL"
pasta_backup_pbix = r"C:\Users\DRT78473\OneDrive - Hospital Albert Einstein\Documentos\DevDesk\Backup\PowerBI"

arquivos_pbix = ["Planejamento.pbix", "NS_analise_Ok.pbix"]

data_formatada = data_atual.strftime("%d-%m-%Y")

for arquivo in arquivos_pbix:
    origem = os.path.join(pasta_origem_pbix, arquivo)
    destino = os.path.join(pasta_backup_pbix, f"{os.path.splitext(arquivo)[0]} - {data_formatada}.pbix")
    
    if os.path.exists(origem):
        shutil.copy2(origem, destino)
        print(f"Backup criado: {destino}")
    else:
        print(f"Arquivo '{arquivo}' não encontrado para backup.")

# 🗑 Remover arquivos antigos do backup
limite_dias = 7
agora = datetime.now()

for arquivo in os.listdir(pasta_backup_pbix):
    caminho_arquivo = os.path.join(pasta_backup_pbix, arquivo)
    if os.path.isfile(caminho_arquivo):
        data_modificacao = datetime.fromtimestamp(os.path.getmtime(caminho_arquivo))
        if (agora - data_modificacao).days > limite_dias:
            os.remove(caminho_arquivo)
            print(f"Arquivo antigo removido: {caminho_arquivo}")

print("✅ Processo concluído com sucesso!")

Arquivo consolidado salvo em: C:\Users\DRT78473\Downloads\scm586g_followup.csv
Arquivo renomeado para: C:\Users\DRT78473\Downloads\scm420g_suggestions.csv
Arquivo movido para: \\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025\Maio\15.05\scm586g_followup.csv
Arquivo movido para: \\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025\Maio\15.05\scm420g_suggestions.csv
Arquivo movido para: \\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025\Maio\15.05\sku.csv
Arquivo movido para: \\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025\Maio\15.05\sku (1).csv
Arquivo de origem movido para: \\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025\Maio\15.05\result (1).csv
Arquivo de origem movido para: \\arqsv0\sl\PlanLogistica\IND

✅ Arquivo final salvo como: C:\Users\DRT78473\OneDrive - Hospital Albert Einstein\Documentos\DevDesk\Codigos\AutomacaoGTPlan\sku 01.04.csv


In [39]:
import pandas as pd
import os
from datetime import datetime

# Caminhos
pasta = r'C:\Users\DRT78473\OneDrive - Hospital Albert Einstein\Documentos\DevDesk\Codigos\AutomacaoGTPlan'
arquivo1 = os.path.join(pasta, 'sku.csv')
arquivo2 = os.path.join(pasta, 'sku (1).csv')

# Pega data de modificação do arquivo
def obter_data_arquivo(caminho):
    ts = os.path.getmtime(caminho)
    dt = datetime.fromtimestamp(ts)
    return dt.strftime('%d.%m'), dt.strftime('%d/%m/%Y')

# Leitura com encoding certo (resolve acentos e BOM)
df1 = pd.read_csv(arquivo1, sep=';', encoding='utf-8-sig', dtype=str)
df2 = pd.read_csv(arquivo2, sep=';', encoding='utf-8-sig', dtype=str)

# Remove cabeçalho duplicado
if list(df2.iloc[0]) == list(df2.columns):
    df2 = df2.iloc[1:].reset_index(drop=True)

# Junta os dois DataFrames
df = pd.concat([df1, df2], ignore_index=True)

# Remove colunas desnecessárias
colunas_remover = [
    'Desc Item', 'Tipo de Material', 'Grupo de Mercadoria',
    'Classificação DRP', 'Motivo', 'Unit x MP', 'Agrupamentos Gerais'
]
df = df.drop(columns=[c for c in colunas_remover if c in df.columns], errors='ignore')

# Renomeia colunas conforme mapeamento
renomear = {
    'Item de Alto Custo (R$)': 'Item_de_Alto_Custo_',
    'Em Falta': 'Em_Falta',
    'Item VMI': 'Item_VMI',
    'Excesso em outro estabelecimento': 'Excesso_em_outro_estabelecimento',
    'Cod Item': 'Cod_Item',
    'Estab.': 'Estab',
    'VL Médio (R$)': 'VL_Medio_',
    '123': 'Curva123',
    'Política': 'Politica',
    'Estoq Disp': 'Estoq_Disp',
    'Estoq Disp (R$)': 'Estoq_Disp_',
    'Estoq Tot': 'Estoq_Tot',
    'Estoq Tot (R$)': 'Estoq_Tot_',
    'Prev Mês': 'Prev_Mes',
    'Mín': 'Minimo',
    'Máx.': 'Maximo',
    'Excesso (R$)': 'Excesso__',
    'Excesso Fut': 'Excesso_Fut',
    'Dias s/ Cons': 'Dias_sem_Cons',
    'Cons 1D': 'Cons_1D',
    'Cons 30D': 'Cons_30D',
    'Média 3M': 'Media_3M',
    'Feb/2025': 'M2',
    'Apr/2025': 'M0',
    'QT Próx OC': 'QT_Prox_OC',
    'DT Próx OC': 'DT_Prox_OC',
    'QT Próx SC': 'QT_Prox_SC',
    'DT Próx SC': 'DT_Prox_SC',
    'QT Próx Sug': 'QT_Prox_Sug',
    'DT Próx Sug': 'DT_Prox_Sug',
    'Últ Fornec': 'ult_Fornec',
    'DT Últ Compra': 'DT_ult_Compra',
    'Cod. Bloqueio': 'Cod_Bloqueio',
    'Min. (Manual)': 'Min_Manual',
    'Min Compl': 'Min_Compl',
    'QT Min Emb': 'QT_Min_Emb',
    'QT Mult Emb': 'QT_Mult_Emb',
    'Depósito padrão de recebimento': 'Deposito_padrao_de_recebimento',
    'Prev Ajus': 'Prev_Ajus',
    'Mín Orig': 'Min_Orig',
    'Máx Orig': 'Max_Orig',
    'Sug Urg': 'Sug_Urg',
    'Sug Plan': 'Sug_Plan',
    'Média 6M': 'Media_6M',
    'Média 12M': 'Media_12M',
    'Min ERP': 'Min_ERP',
    'Máx ERP': 'Max_ERP',
    'PT Ped (ERP)': 'PT_Ped_ERP',
    'Cons Med': 'Cons_Med',
    'Desvios d-1': 'Desvios_d1',
    'Desvio LT': 'Desvio_LT',
    'Apr/2024': 'M12',
    'May/2024': 'M11',
    'Dec/2024': 'M4',
    'QT Reserva': 'QT_Reserva',
    'Lote Min F (Final)': 'Lote_Min_F_Final',
    'Lote Mult F (Final)': 'Lote_Mult_F_Final',
    'QT Transf': 'QT_Transf',
    'QT SC': 'QT_SC',
    'QT OC': 'QT_OC',
    'DT Últ Entr': 'DT_ult_Entr',
    'QT Últ Entr': 'QT_ult_Entr',
    'DT Últ Cons': 'DT_ult_Cons',
    'QT Últ Cons': 'QT_ult_Cons',
    'VL Últ Entr (R$)': 'VL_ult_Entr_',
    'QT Últ Compra': 'QT_ult_Compra',
    'Últ Entr Prev': 'ult_Entr_Prev',
    'LT Plan F': 'LT_Plan_F',
    'LT Compra F': 'LT_Compra_F',
    'LT Fornec F': 'LT_Fornec_F',
    'LT Recb F': 'LT_Recb_F',
    'LT Outro F': 'LT_Outro_F',
    'LT Total': 'LT_Total',
    'Saldo Grupo': 'Saldo_Grupo',
    'Cob Grupo': 'Cob_Grupo',
    'MAPE(%)': 'MAPE',
    'Cobertura Substituto': 'Cobertura_Substituto',
    'Estoque Disp Substituto': 'Estoque_Disp_Substituto',
    'Prev Mês Subsitituto': 'Prev_Mes_Subsitituto',
    'NS M-1': 'NS_M1',
    'NS Alvo': 'NS_Alvo',
    'Tipo de MRP': 'Tipo_de_MRP',
    'Grupo de compras': 'Grupo_de_compras',
    'Centro Abastecedor MRP': 'Centro_Abastecedor_MRP',
    'Ativo ERP': 'Ativo_ERP',
    'Fase Vida': 'Fase_Vida',
    'DT Inclusão': 'DT_Inclusao',
    'Demanda (Manual)': 'Demanda_Manual',
    'Dt Exp Demanda Manual': 'Dt_Exp_Demanda_Manual',
    'LT Plan': 'LT_Plan',
    'LT Compra': 'LT_Compra',
    'LT Fornec': 'LT_Fornec',
    'LT Rec': 'LT_Rec',
    'LT Outros': 'LT_Outros',
    'Política.1': 'Politica2',
    'Calendário.1': 'Calendario2',
    'Lote Min': 'Lote_Min',
    'Lote Mult': 'Lote_Mult',
    'Cod. Item Fornec.': 'Cod_Item_Fornec',
    'Dependência': 'Dependencia',
    'Calendário': 'Calendario',
    'Ponto Pedido': 'Ponto_Pedido',
    'Obs.': 'Obs',
    'Mín ERP': 'Min_ERP'
}

# Ajuste dinâmico com base nos nomes reais das colunas de datas
colunas_data = [col for col in df.columns if '/' in col]
print("\n📅 Colunas com datas detectadas:")
for col in colunas_data:
    print(repr(col))

renomeios_meses = {
    'Mar/2025': 'M1',
    'Feb/2025': 'M2',
    'Jan/2025': 'M3',
    'Dec/2024': 'M4',
    'Nov/2024': 'M5',
    'Oct/2024': 'M6',
    'Sep/2024': 'M7',
    'Aug/2024': 'M8',
    'Jul/2024': 'M9',
    'Jun/2024': 'M10',
    'May/2024': 'M11',
    'Apr/2024': 'M12',
    'Apr/2025': 'M0',
}

renomear.update({k: v for k, v in renomeios_meses.items() if k in df.columns})

# Aplica os renomeios
df.rename(columns=renomear, inplace=True)

# Adiciona Data_Extracao
_, data_extracao = obter_data_arquivo(arquivo1)
df['Data_Extracao'] = data_extracao

# Salva com nome ajustado
data_nome, _ = obter_data_arquivo(arquivo1)
novo_nome = os.path.join(pasta, f'sku {data_nome}.csv')
df.to_csv(novo_nome, index=False, sep=';', encoding='utf-8-sig')

print(f'\n✅ Arquivo final salvo como: {novo_nome}')




📅 Colunas com datas detectadas:
'Dias s/ Cons'
'Feb/2025'
'Mar/2025'
'Apr/2025'
'Apr/2024'
'May/2024'
'Jun/2024'
'Jul/2024'
'Aug/2024'
'Sep/2024'
'Oct/2024'
'Nov/2024'
'Dec/2024'
'Jan/2025'

✅ Arquivo final salvo como: C:\Users\DRT78473\OneDrive - Hospital Albert Einstein\Documentos\DevDesk\Codigos\AutomacaoGTPlan\sku 03.04.csv


In [2]:
import pandas as pd
import glob
import os
import shutil
from datetime import datetime, timedelta

# --- INICIO DO CODIGO B ---
# Caminhos
pasta = os.path.join(os.path.expanduser("~"), "Downloads")
arquivo1 = os.path.join(pasta, 'sku.csv')
arquivo2 = os.path.join(pasta, 'sku (1).csv')

# Pega data de modificação do arquivo
def obter_data_arquivo(caminho):
    ts = os.path.getmtime(caminho)
    dt = datetime.fromtimestamp(ts)
    return dt.strftime('%d.%m'), dt.strftime('%d/%m/%Y')

# Leitura com encoding certo (resolve acentos e BOM)
df1 = pd.read_csv(arquivo1, sep=';', encoding='utf-8-sig', dtype=str)
df2 = pd.read_csv(arquivo2, sep=';', encoding='utf-8-sig', dtype=str)

# Remove cabeçalho duplicado
if list(df2.iloc[0]) == list(df2.columns):
    df2 = df2.iloc[1:].reset_index(drop=True)

# Junta os dois DataFrames
df = pd.concat([df1, df2], ignore_index=True)

# Remove colunas desnecessárias
colunas_remover = [
    'Desc Item', 'Tipo de Material', 'Grupo de Mercadoria',
    'Classificação DRP', 'Motivo', 'Unit x MP', 'Agrupamentos Gerais'
]
df = df.drop(columns=[c for c in colunas_remover if c in df.columns], errors='ignore')

# Renomeia colunas conforme mapeamento
renomear = {
    'Item de Alto Custo (R$)': 'Item_de_Alto_Custo_',
    'Em Falta': 'Em_Falta',
    'Item VMI': 'Item_VMI',
    'Excesso em outro estabelecimento': 'Excesso_em_outro_estabelecimento',
    'Cod Item': 'Cod_Item',
    'Estab.': 'Estab',
    'VL Médio (R$)': 'VL_Medio_',
    '123': 'Curva123',
    'Política': 'Politica',
    'Estoq Disp': 'Estoq_Disp',
    'Estoq Disp (R$)': 'Estoq_Disp_',
    'Estoq Tot': 'Estoq_Tot',
    'Estoq Tot (R$)': 'Estoq_Tot_',
    'Prev Mês': 'Prev_Mes',
    'Mín': 'Minimo',
    'Máx.': 'Maximo',
    'Excesso (R$)': 'Excesso__',
    'Excesso Fut': 'Excesso_Fut',
    'Dias s/ Cons': 'Dias_sem_Cons',
    'Cons 1D': 'Cons_1D',
    'Cons 30D': 'Cons_30D',
    'Média 3M': 'Media_3M',
    'Feb/2025': 'M2',
    'Apr/2025': 'M0',
    'QT Próx OC': 'QT_Prox_OC',
    'DT Próx OC': 'DT_Prox_OC',
    'QT Próx SC': 'QT_Prox_SC',
    'DT Próx SC': 'DT_Prox_SC',
    'QT Próx Sug': 'QT_Prox_Sug',
    'DT Próx Sug': 'DT_Prox_Sug',
    'Últ Fornec': 'ult_Fornec',
    'DT Últ Compra': 'DT_ult_Compra',
    'Cod. Bloqueio': 'Cod_Bloqueio',
    'Min. (Manual)': 'Min_Manual',
    'Min Compl': 'Min_Compl',
    'QT Min Emb': 'QT_Min_Emb',
    'QT Mult Emb': 'QT_Mult_Emb',
    'Depósito padrão de recebimento': 'Deposito_padrao_de_recebimento',
    'Prev Ajus': 'Prev_Ajus',
    'Mín Orig': 'Min_Orig',
    'Máx Orig': 'Max_Orig',
    'Sug Urg': 'Sug_Urg',
    'Sug Plan': 'Sug_Plan',
    'Média 6M': 'Media_6M',
    'Média 12M': 'Media_12M',
    'Min ERP': 'Min_ERP',
    'Máx ERP': 'Max_ERP',
    'PT Ped (ERP)': 'PT_Ped_ERP',
    'Cons Med': 'Cons_Med',
    'Desvios d-1': 'Desvios_d1',
    'Desvio LT': 'Desvio_LT',
    'Apr/2024': 'M12',
    'May/2024': 'M11',
    'Dec/2024': 'M4',
    'QT Reserva': 'QT_Reserva',
    'Lote Min F (Final)': 'Lote_Min_F_Final',
    'Lote Mult F (Final)': 'Lote_Mult_F_Final',
    'QT Transf': 'QT_Transf',
    'QT SC': 'QT_SC',
    'QT OC': 'QT_OC',
    'DT Últ Entr': 'DT_ult_Entr',
    'QT Últ Entr': 'QT_ult_Entr',
    'DT Últ Cons': 'DT_ult_Cons',
    'QT Últ Cons': 'QT_ult_Cons',
    'VL Últ Entr (R$)': 'VL_ult_Entr_',
    'QT Últ Compra': 'QT_ult_Compra',
    'Últ Entr Prev': 'ult_Entr_Prev',
    'LT Plan F': 'LT_Plan_F',
    'LT Compra F': 'LT_Compra_F',
    'LT Fornec F': 'LT_Fornec_F',
    'LT Recb F': 'LT_Recb_F',
    'LT Outro F': 'LT_Outro_F',
    'LT Total': 'LT_Total',
    'Saldo Grupo': 'Saldo_Grupo',
    'Cob Grupo': 'Cob_Grupo',
    'MAPE(%)': 'MAPE',
    'Cobertura Substituto': 'Cobertura_Substituto',
    'Estoque Disp Substituto': 'Estoque_Disp_Substituto',
    'Prev Mês Subsitituto': 'Prev_Mes_Subsitituto',
    'NS M-1': 'NS_M1',
    'NS Alvo': 'NS_Alvo',
    'Tipo de MRP': 'Tipo_de_MRP',
    'Grupo de compras': 'Grupo_de_compras',
    'Centro Abastecedor MRP': 'Centro_Abastecedor_MRP',
    'Ativo ERP': 'Ativo_ERP',
    'Fase Vida': 'Fase_Vida',
    'DT Inclusão': 'DT_Inclusao',
    'Demanda (Manual)': 'Demanda_Manual',
    'Dt Exp Demanda Manual': 'Dt_Exp_Demanda_Manual',
    'LT Plan': 'LT_Plan',
    'LT Compra': 'LT_Compra',
    'LT Fornec': 'LT_Fornec',
    'LT Rec': 'LT_Rec',
    'LT Outros': 'LT_Outros',
    'Política.1': 'Politica2',
    'Calendário.1': 'Calendario2',
    'Lote Min': 'Lote_Min',
    'Lote Mult': 'Lote_Mult',
    'Cod. Item Fornec.': 'Cod_Item_Fornec',
    'Dependência': 'Dependencia',
    'Calendário': 'Calendario',
    'Ponto Pedido': 'Ponto_Pedido',
    'Obs.': 'Obs',
    'Mín ERP': 'Min_ERP'
}

# Ajuste dinâmico com base nos nomes reais das colunas de datas
colunas_data = [col for col in df.columns if '/' in col]
print("\n📅 Colunas com datas detectadas:")
for col in colunas_data:
    print(repr(col))

renomeios_meses = {
    'Mar/2025': 'M1',
    'Feb/2025': 'M2',
    'Jan/2025': 'M3',
    'Dec/2024': 'M4',
    'Nov/2024': 'M5',
    'Oct/2024': 'M6',
    'Sep/2024': 'M7',
    'Aug/2024': 'M8',
    'Jul/2024': 'M9',
    'Jun/2024': 'M10',
    'May/2024': 'M11',
    'Apr/2024': 'M12',
    'Apr/2025': 'M0',
}

renomear.update({k: v for k, v in renomeios_meses.items() if k in df.columns})

# Aplica os renomeios
df.rename(columns=renomear, inplace=True)

# Adiciona Data_Extracao
_, data_extracao = obter_data_arquivo(arquivo1)
df['Data_Extracao'] = data_extracao

# Salva com nome ajustado em Excel
data_nome, _ = obter_data_arquivo(arquivo1)
nome_sku_final = f'sku {data_nome}.xlsx'
caminho_sku_final = os.path.join(pasta, nome_sku_final)
df.to_excel(caminho_sku_final, index=False)

print(f'\n✅ Arquivo final salvo como: {caminho_sku_final}')

# --- FIM DO CODIGO B ---

# AQUI COMEÇA O CÓDIGO A (sem alteração)

# Mapeamento manual dos nomes dos meses em português
meses_corretos = [
    "Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
    "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"
]

# Encontrar todos os arquivos que começam com "result" e terminam em .csv
arquivos = glob.glob(os.path.join(pasta, "result*.csv"))

# Lista para armazenar os DataFrames
dfs = []
for arquivo in arquivos:
    df = pd.read_csv(arquivo, sep=";", dtype=str, encoding="utf-8")
    dfs.append(df)

df_final = pd.concat(dfs, ignore_index=True)
caminho_saida = os.path.join(pasta, "scm586g_followup.csv")
if os.path.exists(caminho_saida):
    os.remove(caminho_saida)
df_final.to_csv(caminho_saida, index=False, sep=";", encoding="utf-8-sig")
print(f"Arquivo consolidado salvo em: {caminho_saida}")

arquivo_antigo = os.path.join(pasta, "mrp8009g_ps_list.csv")
arquivo_novo = os.path.join(pasta, "scm420g_suggestions.csv")
if os.path.exists(arquivo_antigo):
    os.rename(arquivo_antigo, arquivo_novo)
    print(f"Arquivo renomeado para: {arquivo_novo}")
else:
    print("Arquivo 'mrp8009g_ps_list.csv' não encontrado na pasta.")

pasta_destino_base = r"\\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025"
pasta_destino_secundaria = r"\\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN"

arquivos_para_mover = [
    "scm586g_followup.csv",
    "scm420g_suggestions.csv",
    "sku.csv",
    "sku (1).csv",
    nome_sku_final  # Adiciona o arquivo final processado
]

data_atual = datetime.now()
nome_mes = meses_corretos[data_atual.month - 1]

pasta_mes = os.path.join(pasta_destino_base, nome_mes)
if not os.path.exists(pasta_mes):
    os.makedirs(pasta_mes)

pasta_dia = os.path.join(pasta_mes, data_atual.strftime("%d.%m"))
if not os.path.exists(pasta_dia):
    os.makedirs(pasta_dia)

for arquivo in arquivos_para_mover:
    origem = os.path.join(pasta, arquivo)
    destino = os.path.join(pasta_dia, arquivo)
    if os.path.exists(origem):
        shutil.move(origem, destino)
        print(f"Arquivo movido para: {destino}")
    else:
        print(f"Arquivo '{arquivo}' não encontrado na pasta de Downloads.")

for arquivo in arquivos:
    nome_arquivo = os.path.basename(arquivo)
    destino = os.path.join(pasta_dia, nome_arquivo)
    shutil.move(arquivo, destino)
    print(f"Arquivo de origem movido para: {destino}")

for arquivo in arquivos_para_mover:
    origem = os.path.join(pasta_dia, arquivo)
    destino = os.path.join(pasta_destino_secundaria, arquivo)
    if os.path.exists(origem):
        shutil.copy2(origem, destino)
        print(f"Arquivo copiado para: {destino}")
    else:
        print(f"Arquivo '{arquivo}' não encontrado para copiar.")

# Backup de arquivos PBIX
pasta_origem_pbix = r"\\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL"
pasta_backup_pbix = r"C:\Users\DRT78473\OneDrive - Hospital Albert Einstein\Documentos\DevDesk\Backup\PowerBI"

arquivos_pbix = ["Planejamento.pbix", "NS_analise_Ok.pbix"]
data_formatada = data_atual.strftime("%d-%m-%Y")
for arquivo in arquivos_pbix:
    origem = os.path.join(pasta_origem_pbix, arquivo)
    destino = os.path.join(pasta_backup_pbix, f"{os.path.splitext(arquivo)[0]} - {data_formatada}.pbix")
    if os.path.exists(origem):
        shutil.copy2(origem, destino)
        print(f"Backup criado: {destino}")
    else:
        print(f"Arquivo '{arquivo}' não encontrado para backup.")

# Remoção de backups antigos
limite_dias = 7
agora = datetime.now()
for arquivo in os.listdir(pasta_backup_pbix):
    caminho_arquivo = os.path.join(pasta_backup_pbix, arquivo)
    if os.path.isfile(caminho_arquivo):
        data_modificacao = datetime.fromtimestamp(os.path.getmtime(caminho_arquivo))
        if (agora - data_modificacao).days > limite_dias:
            os.remove(caminho_arquivo)
            print(f"Arquivo antigo removido: {caminho_arquivo}")

print("\n✅ Processo concluído com sucesso!")



📅 Colunas com datas detectadas:
'Dias s/ Cons'
'Apr/2025'
'May/2025'
'Jun/2025'
'Jun/2024'
'Jul/2024'
'Aug/2024'
'Sep/2024'
'Oct/2024'
'Nov/2024'
'Dec/2024'
'Jan/2025'
'Feb/2025'
'Mar/2025'

✅ Arquivo final salvo como: C:\Users\DRT78473\Downloads\sku 03.06.xlsx
Arquivo consolidado salvo em: C:\Users\DRT78473\Downloads\scm586g_followup.csv
Arquivo 'mrp8009g_ps_list.csv' não encontrado na pasta.
Arquivo movido para: \\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025\Junho\03.06\scm586g_followup.csv
Arquivo movido para: \\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025\Junho\03.06\scm420g_suggestions.csv
Arquivo movido para: \\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025\Junho\03.06\sku.csv
Arquivo movido para: \\arqsv0\sl\PlanLogistica\INDICADORES\DASHBOARDs\PROJETO GERENCIAL\Extracao GTPLAN\historico_extracao\2025\J

### Automação Extração GTPlan